# Analyzing Thanksgiving Dinner

In this notebook, I'll be working on analyzing data on Thanksgiving dinner in the US. The dataset used contains 1058 responses to an online survey about what Americans eat for Thanksgiving dinner. It consists of 65 columns and 1058 rows, where most of the column names are questions, and most of the column values are string responses to the questions.

To download the dataset, [click here](thanksgiving.csv).

---
#### Loading the dataset

In [1]:
import pandas as pd

data = pd.read_csv("thanksgiving.csv",encoding="Latin-1")

# uncomment the following line to get columns name neatly printed
# cols = [print('=> ',x) for x in data.columns]

##### Filter out records of people who do not celebrate thanksgiving

In [2]:
print("People who don't celebrate it:")
print(data["Do you celebrate Thanksgiving?"].value_counts())

#Filter out those with "No" response
data = data[data["Do you celebrate Thanksgiving?"] == "Yes"]

People who don't celebrate it:
Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64


---
##### Explore Main Dishes

In [3]:
print("Main dishes count:")
print(data["What is typically the main dish at your Thanksgiving dinner?"].value_counts())

Main dishes count:
Turkey                    859
Other (please specify)     35
Ham/Pork                   29
Tofurkey                   20
Chicken                    12
Roast beef                 11
I don't know                5
Turducken                   3
Name: What is typically the main dish at your Thanksgiving dinner?, dtype: int64


---
##### Explore "Do you typically have gravy?" column

In [4]:
# Let's filter only "Tofurkey" dish 

gravy_Tofurkey = data[data["What is typically the main dish at your Thanksgiving dinner?"] == "Tofurkey"]["Do you typically have gravy?"]

gravy_Tofurkey.value_counts()

Yes    12
No      8
Name: Do you typically have gravy?, dtype: int64

---
##### Figuring Out What Pies People Eat

In [5]:
apple_isnull = data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Apple"].isnull()
pumpkin_isnull = data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pumpkin"].isnull()
pecan_isnull = data["Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pecan"].isnull()

print("Apple: ",apple_isnull[apple_isnull == False].count())
print("Pumpkin: ", pumpkin_isnull[pumpkin_isnull == False].count())
print("Pecan: ",pecan_isnull[pecan_isnull == False].count())

ate_pies = apple_isnull & pumpkin_isnull & pecan_isnull

print()

print("Total who ate at least one: ",ate_pies[ate_pies == False].count())
print("Total who ate none: ",ate_pies[ate_pies == True].count())

Apple:  514
Pumpkin:  729
Pecan:  342

Total who ate at least one:  876
Total who ate none:  104


---
##### Converting Age To Numeric

In [6]:
import math

def conv(input_str):
    if pd.isnull(input_str):
        return None
    return int(input_str.split(" ")[0].replace("+",""))

data["age_int"] = data["Age"].apply(conv)

data["age_int"].describe()

/Users/iHijazi/anaconda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


count    947.000000
mean      40.089757
std       15.352014
min       18.000000
25%             NaN
50%             NaN
75%             NaN
max       60.000000
Name: age_int, dtype: float64

---
##### Getting Income Average 

In [7]:
def conv_income(input_str):
    if pd.isnull(input_str) or input_str == "Prefer not to answer":
        return None
    elif input_str == "$200,000 and up":
        return 200000
    splitted = input_str.replace("$","").replace(" to","").replace(",","").split(" ")
    avg = (int(splitted[0]) + int(splitted[1])) / 2
    return avg

data["income_int"] = data["How much total combined money did all members of your HOUSEHOLD earn last year?"].apply(conv_income)

data["income_int"].describe()

/Users/iHijazi/anaconda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


count       829.000000
mean      86486.276840
std       57789.467567
min        4999.500000
25%                NaN
50%                NaN
75%                NaN
max      200000.000000
Name: income_int, dtype: float64

---
##### Correlating Travel Distance And Income

In [8]:
print("How far people average earning under 50000 will travel?")
print("----------------------------------------------------------------------------------------")
print(data["How far will you travel for Thanksgiving?"][data["income_int"] < 50000].value_counts())
print("----------------------------------------------------------------------------------------")
print("How far people average earning over 150000 will travel?")
print(data["How far will you travel for Thanksgiving?"][data["income_int"] > 150000].value_counts())

How far people average earning under 50000 will travel?
----------------------------------------------------------------------------------------
Thanksgiving is happening at my home--I won't travel at all                         106
Thanksgiving is local--it will take place in the town I live in                      92
Thanksgiving is out of town but not too far--it's a drive of a few hours or less     64
Thanksgiving is out of town and far away--I have to drive several hours or fly       16
Name: How far will you travel for Thanksgiving?, dtype: int64
----------------------------------------------------------------------------------------
How far people average earning over 150000 will travel?
Thanksgiving is happening at my home--I won't travel at all                         66
Thanksgiving is local--it will take place in the town I live in                     34
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    25
Thanksgiving is out of town and far

---
##### Linking Friendship And Age

In [9]:
data.pivot_table(index="Have you ever tried to meet up with hometown friends on Thanksgiving night?",
                 columns='Have you ever attended a "Friendsgiving?"',values="age_int")

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,42.283702,37.010526
Yes,41.475410,33.976744


In [10]:
data.pivot_table(index="Have you ever tried to meet up with hometown friends on Thanksgiving night?",
                 columns='Have you ever attended a "Friendsgiving?"',values="income_int")

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,89456.824480,83124.552632
Yes,89553.119048,76315.319079


##### Most common dessert people eat

In [11]:
print("Apple cobbler: ",data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Apple cobbler"][data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Apple cobbler"].isnull() == False].count())
print("Blondies: ",data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Blondies"][data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Blondies"].isnull() == False].count())
print("Brownies: ",data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Brownies"][data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Brownies"].isnull() == False].count())
print("Carrot cake: ",data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Carrot cake"][data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Carrot cake"].isnull() == False].count())
print("Cheesecake: ",data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Cheesecake"][data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Cheesecake"].isnull() == False].count())
print("Cookies: ",data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Cookies"][data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Cookies"].isnull() == False].count())
print("Fudge: ",data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Fudge"][data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Fudge"].isnull() == False].count())
print("Ice cream: ",data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Ice cream"][data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Ice cream"].isnull() == False].count())
print("Peach cobbler: ",data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Peach cobbler"][data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Peach cobbler"].isnull() == False].count())
print("None: ",data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - None"][data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - None"].isnull() == False].count())
print("Other: ",data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Other (please specify)"][data["Which of these desserts do you typically have at Thanksgiving dinner? Please select all that apply.   - Other (please specify)"].isnull() == False].count())

Apple cobbler:  110
Blondies:  16
Brownies:  128
Carrot cake:  72
Cheesecake:  191
Cookies:  204
Fudge:  43
Ice cream:  266
Peach cobbler:  103
None:  295
Other:  134


##### How many people work on Black Friday?

In [12]:
data["Will you employer make you work on Black Friday?"].value_counts()

Yes              43
No               20
Doesn't apply     7
Name: Will you employer make you work on Black Friday?, dtype: int64

---
---
# That's it

Many more questions can be asked, like:
- Find regional patterns in the dinner menus.
- Find age, gender, and income based patterns in dinner menus.
- Figure out the most common complete meal people eat.

Why don't you try it yourself?

Prepared by Issam Hijazi

https://ae.linkedin.com/in/ihijazi

@iHijazi